In [ ]:
import torch as th 
th.backends.cudnn.benchmark = True

In [ ]:
from mainNet import Decoder_Train_Net

train_net = Decoder_Train_Net(True).cuda().train()

In [ ]:
from dataloader import ImageLoder
from torch.utils.data import DataLoader

dataset = DataLoader(dataset= ImageLoder(r"./data\train2014", 1000), 
                     batch_size= 1, 
                     shuffle= True, 
                     num_workers= 4
)

In [ ]:
optimizer = th.optim.Adam(train_net.decoder.parameters(), lr= 0.001)

In [ ]:
scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min', 
                                                    factor= 0.9, patience= 10, 
                                                    verbose= True, min_lr= 1e-4)

In [ ]:
from lossFunction import Decoder_Train_Loss

loss_fn = Decoder_Train_Loss().cuda()

In [ ]:
epochs = 1

In [ ]:
now_epoch: int = 1

In [ ]:
train_net.decoder.load_state_dict(th.load(r"./pretrained_models\decoder0.8_alpha.pth"))

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter(utils.get_logPath(r"./logs"))

In [ ]:
for _ in range(epochs): 
    loss_all = 0 
    print("epoch: {:d}".format(now_epoch))
    for img in dataset:
        img = img.cuda()
        train_net(img)
        
        loss = loss_fn(train_net.dlist ,train_net.elist)
        
        loss_all += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # scheduler.step(loss_all)
    # writer.add_scalar("loss", loss_all, now_epoch)
    print("loss: ", loss_all)
    now_epoch += 1

In [ ]:
for name, parms in train_net.decoder.named_parameters():
	print('-->name:', name, '--weight', th.mean(parms.data), ' -->grad_value:', th.mean(parms.grad))

In [ ]:
th.save(train_net.decoder.state_dict(), r"./pretrained_models/decoder0.4.pth")